In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
!pip install -q git+https://github.com/tensorflow/docs
from  IPython import display
import pathlib
import shutil
import tempfile
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
print(tf.__version__)

2.1.0


In [2]:
#Taken from tensorflow site, not sure of purpose
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [3]:
#Takes in dataset
f = h5py.File('real-xy-labeled-images.h5', 'r')


In [4]:
#Shows what is stored in the dataset
list(f.keys())

['max_charge',
 'test_features',
 'test_targets',
 'train_features',
 'train_targets']

In [5]:
#Shows what is max charge
f['max_charge'][0]

3.6127838567197355

In [6]:
#Shows what each data point looks like 
f['test_features'].shape

(538, 128, 128, 3)

In [7]:
#Converts and Splits data set into training, validation and test sets and makes them arrays
train_images = np.array(f['train_features'][0:1600])  
train_labels = np.array(f['train_targets'][0:1600]) 
validation_images = np.array(f['train_features'][1599:2150])
validation_labels = np.array(f['train_targets'][1599:2150])
test_images = np.array(f['test_features'][:]) 
test_labels = np.array(f['test_targets'][:]) 

In [ ]:
train_images.shape

In [ ]:
validation_images.shape

In [ ]:
validation_labels.shape

In [ ]:
f['train_features'].shape

In [ ]:
#Example of label 0
plt.imshow(f['test_features'][2])

In [ ]:
#Example of label 1
plt.imshow(f['test_features'][5])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(f['train_features'][i], cmap=plt.cm.binary)
    plt.xlabel(f['train_targets'][i])
plt.show()

In [ ]:
N_VALIDATION = 551
N_TRAIN = 1600
BUFFER_SIZE = 100
BATCH_SIZE = 400
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
#Copied from Tensorflow, example of learning rate decearsing over time 
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

In [ ]:
#Compiles and Stores Iteration results into History
def compile_and_fit(model, name, max_epochs):
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.summary()
    
    history = model.fit(train_images, train_labels, 
                         batch_size = 200, 
                         epochs=max_epochs, 
                         validation_data = (validation_images,validation_labels)
                       )
    return history

In [ ]:
tiny_model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128,3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3)
])

In [ ]:
size_histories = {}

In [ ]:
tiny_history = compile_and_fit(tiny_model, 'sizes/Tiny',20) 

In [ ]:
print(tiny_history.get('accuracy'))


In [ ]:
 t = np.arange(0,20)
 a = history.history.get('accuracy')
 plt.plot(t,a)
 plt.show()   

In [ ]:
tiny_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = tiny_model.fit(train_images, train_labels, 
                         batch_size = 500, 
                         epochs=20, 
                         validation_data = (validation_images,validation_labels))

print('\nhistory dict:', history.history)

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(history)
plt.ylim([0.5, 0.7])

In [ ]:
test_loss, test_acc = tiny_model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
test_labels[0]

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

In [ ]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

In [ ]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [ ]:
np.argmax(predictions_single[0])